In [1]:
import tensorflow as tf

class NaiveDense:

    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

In [2]:
class NaiveSequential:

    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights

In [3]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [4]:
class BatchGenerator:

    def __init__(self, images, labels, batch_size=128):
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

The most difficult part of the process is the “training step”: updating the weights of the model after running it on one batch of data. We need to:

* Compute the predictions of the model for the images in the batch
* Compute the loss value for these predictions given the actual labels
* Compute the gradient of the loss with regard to the model’s weights
* Move the weights by a small amount in the direction opposite to the gradient

In [5]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
      predictions = model(images_batch)
      per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
          labels_batch, predictions)
      average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [14]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [13]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, model.weights):
        w.assign_sub(w * learning_rate)

In [15]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
      print('Epoch %d' % epoch_counter)
      batch_generator = BatchGenerator(images, labels)
      for batch_counter in range(len(images) // batch_size):
          images_batch, labels_batch = batch_generator.next()
          loss = one_training_step(model, images_batch, labels_batch)
          if batch_counter % 100 == 0:
              print('loss at batch %d: %.2f' % (batch_counter, loss))

In [16]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 2.30
loss at batch 100: 2.30
loss at batch 200: 2.30
loss at batch 300: 2.30
loss at batch 400: 2.30
Epoch 1
loss at batch 0: 2.30
loss at batch 100: 2.30
loss at batch 200: 2.30
loss at batch 300: 2.30
loss at batch 400: 2.30
Epoch 2
loss at batch 0: 2.30
loss at batch 100: 2.30
loss at batch 200: 2.30
loss at batch 300: 2.30
loss at batch 400: 2.29
Epoch 3
loss at batch 0: 2.29
loss at batch 100: 2.30
loss at batch 200: 2.30
loss at batch 300: 2.29
loss at batch 400: 2.29
Epoch 4
loss at batch 0: 2.29
loss at batch 100: 2.29
loss at batch 200: 2.29
loss at batch 300: 2.29
loss at batch 400: 2.29
Epoch 5
loss at batch 0: 2.28
loss at batch 100: 2.29
loss at batch 200: 2.29
loss at batch 300: 2.28
loss at batch 400: 2.28
Epoch 6
loss at batch 0: 2.27
loss at batch 100: 2.28
loss at batch 200: 2.28
loss at batch 300: 2.27
loss at batch 400: 2.27
Epoch 7
loss at batch 0: 2.26
loss at batch 100: 2.27
loss at batch 200: 2.27
loss at batch 300: 2.25
loss at batch 40

In [17]:
import numpy as np
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print('accuracy: %.2f' % matches.mean())

accuracy: 0.20


In [18]:
input_var = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
   result = tf.square(input_var)
gradient = tape.gradient(result, input_var)

In [19]:
time = tf.Variable(0.)
with tf.GradientTape() as outer_tape:
  with tf.GradientTape() as inner_tape:
    position =  4.9 * time ** 2
  speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)